# Deutsch-Jozsa Algorithm Implementation

Exploring quantum advantage through the Deutsch-Jozsa algorithm using Qiskit.

In [1]:
# ============================================================================
# IMPORTS - All package imports for this notebook
# ============================================================================
import random
import numpy as np
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

print("All packages imported successfully!")
print(f"Qiskit version: {qiskit.__version__}")

All packages imported successfully!
Qiskit version: 2.3.0


## Problem 1: Generating Random Boolean Functions

### Introduction

The Deutsch-Jozsa algorithm works with Boolean functions that take a fixed number of binary inputs and return a single binary output. Each function is guaranteed to be either constant (always returns the same value) or balanced (returns True for exactly half of all possible inputs). For four Boolean inputs, there are 2⁴ = 16 possible input combinations. A constant function returns the same output for all 16 inputs, while a balanced function returns True for exactly 8 inputs and False for the other 8.

### Solution Approach

We implement a function that randomly selects between generating a constant or balanced function. For constant functions, we randomly choose to return either always True or always False. For balanced functions, we randomly select exactly 8 of the 16 possible input combinations to return True, with the remaining 8 returning False. The function returns a callable Python function object that can be queried with any four Boolean values as arguments.

### References

**IBM Quantum Learning - Deutsch-Jozsa Algorithm**
- URL: https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa
- Relevance: Official documentation explaining the Deutsch-Jozsa algorithm and the types of Boolean functions it operates on.

### Helper Functions

Before implementing the main function generator, we create a helper function to generate all possible Boolean input combinations. For n Boolean inputs, there are 2^n possible combinations. This helper function systematically generates all combinations by converting integers from 0 to 2^n-1 into their binary representations. Each bit in the binary number becomes a Boolean value in the input tuple. This approach is cleaner and more flexible than hardcoding all 16 combinations, and can easily scale to different numbers of inputs if needed.

In [2]:

# PROBLEM 1: Helper Functions


def generate_all_inputs(n):
    """
    Generate all possible n-bit Boolean input combinations.
    
    For n Boolean inputs, there are 2^n possible combinations.
    
    Args:
        n: Number of Boolean inputs (e.g., 4 for four-bit inputs)
    
    Returns:
        list: List of tuples, each containing n Boolean values
    
    Example:
        >>> generate_all_inputs(2)
        [(False, False), (False, True), (True, False), (True, True)]
    """
    # Generate all integers from 0 to 2^n - 1
    num_combinations = 2 ** n
    all_combinations = []
    
    for i in range(num_combinations):
        # Convert integer to binary representation
        # Extract each bit as a Boolean value
        combination = tuple(bool(i & (1 << bit)) for bit in range(n))
        all_combinations.append(combination)
    
    return all_combinations

# Test the helper function
print("Testing generate_all_inputs():")
print(f"2-bit inputs: {generate_all_inputs(2)}")
print(f"3-bit inputs: {len(generate_all_inputs(3))} combinations")
print(f"4-bit inputs: {len(generate_all_inputs(4))} combinations (used in this problem)")

Testing generate_all_inputs():
2-bit inputs: [(False, False), (True, False), (False, True), (True, True)]
3-bit inputs: 8 combinations
4-bit inputs: 16 combinations (used in this problem)


In [3]:

# PROBLEM 1: Generating Random Boolean Functions


def random_constant_balanced():
    """
    Generates a random Boolean function that is either constant or balanced.
    
    The function takes four Boolean arguments and returns a single Boolean output.
    - Constant: Always returns the same value (all True or all False)
    - Balanced: Returns True for exactly half (8 out of 16) of the input combinations
    
    Returns:
        function: A callable that takes 4 Boolean arguments and returns a Boolean
    """
    
    # Randomly decide if the function will be constant or balanced
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Constant function: randomly choose to return all True or all False
        constant_value = random.choice([True, False])
        
        def constant_function(a, b, c, d):
            """Constant function - always returns the same value."""
            return constant_value
        
        return constant_function
    
    else:
        # Balanced function: True for exactly 8 of 16 possible inputs
        # Generate all 16 possible input combinations
        all_inputs = generate_all_inputs(4)  
        
        # Randomly select exactly 8 inputs that will return True
        true_inputs = set(random.sample(all_inputs, 8))
        
        def balanced_function(a, b, c, d):
            """Balanced function - returns True for exactly 8 input combinations."""
            return (a, b, c, d) in true_inputs
        
        return balanced_function

print("random_constant_balanced() function created successfully!")

random_constant_balanced() function created successfully!


### Testing the Random Function Generator

We test the generator by creating several random functions and verifying they meet the constant or balanced criteria. For each generated function, we test it against all 16 possible input combinations and count how many return True. A constant function should return True either 0 times (always False) or 16 times (always True). A balanced function should return True exactly 8 times.

In [4]:

# Testing random_constant_balanced()


# Generate all 16 possible input combinations using helper function
all_inputs = generate_all_inputs(4)

print("=" * 70)
print("Testing random_constant_balanced() Function")
print("=" * 70)

# Test multiple generated functions
for i in range(5):
    print(f"\n--- Test {i+1} ---")
    
    # Generate a random function
    f = random_constant_balanced()
    
    # Test it on all 16 inputs
    true_count = sum(1 for inputs in all_inputs if f(*inputs))
    
    # Determine type based on true_count
    if true_count == 0 or true_count == 16:
        func_type = "Constant"
        value = "always False" if true_count == 0 else "always True"
        print(f"Function type: {func_type} ({value})")
    elif true_count == 8:
        func_type = "Balanced"
        print(f"Function type: {func_type}")
        print(f"Returns True for 8/16 inputs, False for 8/16 inputs")
    else:
        func_type = "ERROR"
        print(f"ERROR: Function returned True for {true_count}/16 inputs")
    
    print(f"True count: {true_count}/16")

print("\n" + "=" * 70)
print("All tests completed!")
print("=" * 70)

Testing random_constant_balanced() Function

--- Test 1 ---
Function type: Constant (always True)
True count: 16/16

--- Test 2 ---
Function type: Balanced
Returns True for 8/16 inputs, False for 8/16 inputs
True count: 8/16

--- Test 3 ---
Function type: Balanced
Returns True for 8/16 inputs, False for 8/16 inputs
True count: 8/16

--- Test 4 ---
Function type: Balanced
Returns True for 8/16 inputs, False for 8/16 inputs
True count: 8/16

--- Test 5 ---
Function type: Constant (always True)
True count: 16/16

All tests completed!


### Conclusion

Problem 1 successfully implements a generator for random Boolean functions that are either constant or balanced. The function works by first randomly deciding between the two types, then implementing the appropriate logic. For constant functions, we simply return the same Boolean value for all inputs. For balanced functions, we randomly select exactly 8 of the 16 possible input combinations to return True. Testing confirms that all generated functions correctly satisfy either the constant property (returning the same value for all 16 inputs) or the balanced property (returning True for exactly 8 inputs and False for the other 8). These functions will serve as test cases for the classical and quantum algorithms in subsequent problems.

## Problem 2: Classical Testing for Function Type

### Introduction

Before exploring quantum approaches, we must understand the classical solution to determine whether a Boolean function is constant or balanced. Classically, this requires querying the function multiple times to observe its behavior across different inputs. For a function with n input bits, there are 2^n possible input combinations. In the worst case, we need to query more than half of these inputs to be absolutely certain whether the function is constant or balanced. For our four-bit functions (16 possible inputs), we need to check at least 9 inputs to guarantee 100% certainty.

### Solution Approach

The classical algorithm works by systematically testing the function with different inputs and counting how many return True. We continue until we can definitively conclude the function type. If we find both True and False outputs, we know immediately it's balanced (since constant functions return the same value for all inputs). Otherwise, we must test enough inputs to be certain. For maximum efficiency, we test inputs until either: (1) we find both True and False (proving it's balanced), or (2) we've tested more than half the inputs with the same result (proving it's constant).

### References

**IBM Quantum Learning - Deutsch Algorithm**
- URL: https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm
- Relevance: Explains the classical problem complexity that quantum algorithms aim to improve upon.

**Computational Complexity Theory**
- URL: https://plato.stanford.edu/entries/computational-complexity/
- Relevance: Provides context for understanding query complexity and why the number of function calls matters in algorithm analysis.

In [ ]:

# PROBLEM 2: Classical Testing for Function Type


def determine_constant_balanced(f):
    """
    Classically determine if a Boolean function is constant or balanced.
    
    Tests the function with different inputs until we can definitively
    determine its type. Returns as soon as certainty is achieved.
    
    Args:
        f: A Boolean function taking 4 Boolean arguments
    
    Returns:
        str: Either "constant" or "balanced"
    """
    # Generate all possible 4-bit input combinations
    all_inputs = generate_all_inputs(4)
    
    # Track results
    true_count = 0
    false_count = 0
    queries = 0
    
    # Test inputs until we can determine the function type
    for inputs in all_inputs:
        queries += 1
        result = f(*inputs)
        
        if result:
            true_count += 1
        else:
            false_count += 1
        
        # Early termination: if we find both True and False, it's balanced
        if true_count > 0 and false_count > 0:
            print(f"Determined after {queries} queries: Found both True and False")
            return "balanced"
        
        # If we've tested more than half (9+ inputs) with same result, it's constant
        if queries > 8:
            if true_count == queries:
                print(f"Determined after {queries} queries: All outputs are True")
                return "constant"
            elif false_count == queries:
                print(f"Determined after {queries} queries: All outputs are False")
                return "constant"
    
    # Fallback: tested all inputs
    if true_count == 16 or false_count == 16:
        print(f"Determined after {queries} queries: All outputs identical")
        return "constant"
    else:
        print(f"Determined after {queries} queries: Exactly half True, half False")
        return "balanced"

print("determine_constant_balanced() function created successfully!")

determine_constant_balanced() function created successfully!
